In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import random

sys.path.append('../')

from src.data.data_utils import *
from src.data.dataset import *

ModuleNotFoundError: No module named 'data'

In [51]:
data_root = '/u/ayang1/foundm/scratch_sc/data/denmark_eurocrop'
denmark_ts = pd.read_csv(os.path.join(data_root, '2018/csvs/S1_point_10days_500m_1Jan-31Dec_Denmark_marker.csv'))
denmark_ts['Afgroede'] = denmark_ts['Afgroede'].replace(np.nan, 'None') 
denmark_ts = denmark_ts.drop('system:index', axis=1)

hcat_translation = pd.read_csv(os.path.join(data_root, '2019/eurocrop_2019_translations.csv'))

def translate_codes(row):
    row = hcat_translation.loc[hcat_translation['original_code'] == row['Afgkode']]
    if len(row) == 0:
        return 'None', 'None'
    row = row.iloc[0]
    return np.array([row['HCAT3_code'], row['HCAT3_name']])

translated = denmark_ts.apply(translate_codes, axis=1)
denmark_ts['HCAT3_code'] = translated.apply(lambda x: x[0])
denmark_ts['HCAT3_name'] = translated.apply(lambda x: x[1])

In [52]:
np.unique(denmark_ts['Afgkode'].values)

array([  0,   1,   2,   3,   4,   5,   6,   7,  10,  11,  14,  15,  16,
        21,  22,  30,  31,  32,  56, 101, 102, 120, 124, 150, 151, 152,
       153, 160, 171, 173, 210, 212, 214, 215, 216, 230, 234, 247, 250,
       251, 252, 254, 255, 256, 257, 260, 263, 264, 266, 267, 268, 269,
       270, 271, 276, 278, 280, 286, 308, 310, 311, 312, 314, 317, 318,
       319, 325, 327, 339, 411, 488, 497, 502, 513, 514, 515, 578, 580,
       583, 585, 586, 587, 592, 593, 602, 701, 702, 703, 704, 706, 707,
       709, 907])

In [28]:
denmark_ts.head()

,Afgkode,Afgroede,IMK_areal,VH_20180101,VH_20180111,VH_20180121,VH_20180131,VH_20180210,VH_20180220,VH_20180302,...,VV_20181018,VV_20181028,VV_20181107,VV_20181117,VV_20181127,VV_20181207,VV_20181217,.geo,HCAT3_code,HCAT3_name
0,0,None,3.59,-17.707537,-18.155216,-18.010496,-19.301693,-18.351768,-20.883432,-19.158804,...,-11.762968,-11.176981,-10.830946,-11.441194,-11.250573,-11.305936,-11.642761,"{""type"":""MultiPoint"",""coordinates"":[]}",None,None
1,0,None,3.66,-17.148403,-17.113312,-17.093094,-18.626764,-17.293833,-20.220556,-18.771097,...,-11.864050,-11.011982,-10.125316,-11.580354,-10.956139,-11.047517,-11.574070,"{""type"":""MultiPoint"",""coordinates"":[]}",None,None
2,247,"Milj�gr�s MVJ-tilsagn (0 N), omdrift",4.93,-15.169315,-15.369123,-14.556433,-15.194209,-15.157029,-16.534246,-15.935643,...,-10.822327,-10.472769,-9.486685,-10.988078,-10.227196,-10.072080,-10.432675,"{""type"":""MultiPoint"",""coordinates"":[]}",3302000000,pasture_meadow_grassland_grass
3,247,"Milj�gr�s MVJ-tilsagn (0 N), omdrift",9.83,-16.663319,-17.057844,-16.285580,-17.037289,-17.247458,-18.712267,-18.248140,...,-11.530586,-11.084849,-10.546886,-11.204223,-11.033155,-11.003602,-11.486580,"{""type"":""MultiPoint"",""coordinates"":[]}",3302000000,pasture_meadow_grassland_grass
4,247,"Milj�gr�s MVJ-tilsagn (0 N), omdrift",9.83,-14.610826,-15.187419,-14.881934,-15.303130,-15.200989,-17.115461,-16.309921,...,-9.944876,-10.073216,-9.254470,-10.071044,-10.093740,-10.155833,-9.837563,"{""type"":""MultiPoint"",""coordinates"":[]}",3302000000,pasture_meadow_grassland_grass


In [29]:
labels, counts = np.unique(denmark_ts['HCAT3_name'], return_counts=True)
to_drop = []
for i in range(len(labels)):
    if counts[i] <= 25:
        to_drop.append(labels[i])
to_drop

['None',
 'alfalfa_lucerne',
 'blackcurrant_cassis',
 'clover',
 'flowers_ornamental_plants',
 'nurseries_nursery',
 'onions',
 'other_arable_land_crops',
 'populus',
 'redcurrant',
 'sod_turf',
 'spinach',
 'spring_rapeseed_rape',
 'spring_rye',
 'spring_triticale',
 'spring_unspecified_cereals',
 'strawberries',
 'sugar_beet',
 'sweet_lupins',
 'willows_osiers',
 'winter_triticale']

In [30]:
print(f'predrop len: {len(denmark_ts)}')

labels, counts = np.unique(denmark_ts['HCAT3_name'], return_counts=True)
to_drop = []

for i in range(len(labels)):
    if counts[i] <= 25:
        to_drop.append(labels[i])

print(f'to_drop {to_drop}')

row_mask = [False if x in to_drop else True for x in denmark_ts['HCAT3_name']]
denmark_ts = denmark_ts.iloc[row_mask, :]

print(f'postdrop len: {len(denmark_ts)}')

predrop len: 7144
to_drop ['None', 'alfalfa_lucerne', 'blackcurrant_cassis', 'clover', 'flowers_ornamental_plants', 'nurseries_nursery', 'onions', 'other_arable_land_crops', 'populus', 'redcurrant', 'sod_turf', 'spinach', 'spring_rapeseed_rape', 'spring_rye', 'spring_triticale', 'spring_unspecified_cereals', 'strawberries', 'sugar_beet', 'sweet_lupins', 'willows_osiers', 'winter_triticale']
postdrop len: 7037


In [31]:
generate_s1_dataset(
    denmark_ts, 
    './',
    'test',
    columns = ['vh', 'vv', 'label'],
    indicies = [(3, 39), (39, 75), 'HCAT3_name']
)

In [9]:
ds = CropTypeDataset('./test.h5', bands = ['vh', 'vv'])

In [55]:
import random

codes = [0,   1,   2,   3,   4,   5,   6,   7,  10,  11,  14,  15,  16,
       21,  22,  30,  31,  32,  56, 101, 102, 120, 124, 150, 151, 152,
       153, 160, 171, 173, 210, 212, 214, 215, 216, 230, 234, 247, 250,
       251, 252, 254, 255, 256, 257, 260, 263, 264, 266, 267, 268, 269,
       270, 271, 276, 278, 280, 286, 308, 310, 311, 312, 314, 317, 318,
       319, 325, 327, 339, 411, 488, 497, 502, 513, 514, 515, 578, 580,
       583, 585, 586, 587, 592, 593, 602, 701, 702, 703, 704, 706, 707,
       709, 907]
codes = [str(x) for x in codes]

colors = ['#' + ''.join(random.choices('0123456789ABCDEF', k=6)) for _ in range(39)]

In [56]:
import pprint
pprint.pprint(dict(zip(codes, colors)))

{'0': '#05E481',
 '1': '#A891FF',
 '10': '#69CE4C',
 '101': '#78D316',
 '102': '#D0B2A3',
 '11': '#CCFAAC',
 '120': '#0B80D3',
 '124': '#A2E8C8',
 '14': '#423325',
 '15': '#ADD40B',
 '150': '#724C1B',
 '151': '#38F95B',
 '152': '#14C175',
 '153': '#984A6D',
 '16': '#F04AB6',
 '160': '#DA58C0',
 '171': '#EA979C',
 '173': '#E29A70',
 '2': '#8E5DF3',
 '21': '#5D98FC',
 '210': '#E70912',
 '212': '#BFE895',
 '214': '#5A930F',
 '215': '#3DE31D',
 '216': '#3CC585',
 '22': '#8B4756',
 '230': '#442781',
 '234': '#B79373',
 '247': '#CF5BE4',
 '250': '#47C1AD',
 '3': '#2086FF',
 '30': '#03F8F3',
 '31': '#5C21E1',
 '32': '#3A8950',
 '4': '#58B3BD',
 '5': '#4D2BB7',
 '56': '#612C94',
 '6': '#151FEB',
 '7': '#C70A58'}


In [ ]:
len(pd.read_csv(''))